In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score, roc_curve, f1_score, recall_score

In [2]:
# Directory for Colab (optional, use if running Colab)
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import os
os.chdir('/content/gdrive/MyDrive/NLP_project')

In [4]:
!pwd

/content/gdrive/MyDrive/NLP_project


In [5]:
def path_results(transformers, attack):
  results_filename = 'results/imdb_results_' + transformers + '_' + attack + '.pickle'
  return results_filename

In [6]:
def compute_roc_auc(df_results, method):
  '''Input : df_results
  method : 'mcd' or 'cov' '''
  assert method=='mcd' or method=='cov'
  if method=='mcd':
    return roc_auc_score(df_results['adversarial'], df_results['OL_mcd'])
  return roc_auc_score(df_results['adversarial'], df_results['OL_cov'])

In [7]:
def get_threshold(df_results, method, fixed_fpr=0.1):
  '''Input : df_results
  method : 'mcd' or 'cov' 
  fixed_fpr: 0.1 as default to compute f1_score and recall'''
  assert method=='mcd' or method=='cov'
  if method=='mcd':
    y_score = df_results['OL_mcd']
    fpr_array, tpr_array, threshold_array = roc_curve(df_results['adversarial'],  df_results['OL_mcd'])
  else:
    y_score = df_results['OL_cov']
    fpr_array, tpr_array, threshold_array = roc_curve(df_results['adversarial'],  df_results['OL_cov'])

  i = 0
  fpr = 0
  while fpr<=fixed_fpr:
    i+=1
    fpr = fpr_array[i]

  return threshold_array[i]

In [8]:
def compute_f1_score(df_results, method, fixed_fpr=0.1):
  threshold = get_threshold(df_results, method, fixed_fpr)
  if method=='mcd':
    y_pred = df_results['OL_mcd']>=threshold - 1e-99
  else:
    y_pred = df_results['OL_cov']>=threshold - 1e-99
  return f1_score(df_results['adversarial'], y_pred)

In [9]:
def compute_recall_score(df_results, method, fixed_fpr=0.1):
  threshold = get_threshold(df_results, method, fixed_fpr)
  if method=='mcd':
    y_pred = df_results['OL_mcd']>=threshold - 1e-99
  else:
    y_pred = df_results['OL_cov']>=threshold - 1e-99
  return recall_score(df_results['adversarial'], y_pred)

In [10]:
def compute_metrics(transformers, attack, method, fixed_fpr=0.1):
  '''Input : 
  transformers in {'bert', 'roberta}
  attack in {'textfooler', 'pwws', 'bae', 'tf-adj'}
  method in {'mcd', 'cov'}
  Output : list of 3 metrics : AUC, F1, recall'''
  results_filename = path_results(transformers, attack)
  df_results = pd.read_pickle(results_filename)

  L = []
  auc = compute_roc_auc(df_results, method)
  L.append(auc)
  f1_score = compute_f1_score(df_results, method, fixed_fpr)
  L.append(f1_score)
  recall_score = compute_recall_score(df_results, method, fixed_fpr)
  L.append(recall_score)
  return L

In [11]:
def complete_results():
  list_idx = []
  metrics_matrix = []
  
  for transformers in ['bert', 'roberta']:
    for attack in ['textfooler', 'pwws', 'bae', 'tf-adj']:
      for method in ['mcd', 'cov']:
        list_idx.append(transformers + '_' + attack + '_' + method)
        metrics_matrix.append(compute_metrics(transformers, attack, method))
  
  df = pd.DataFrame(metrics_matrix, index=list_idx, columns=['AUC', 'F1', 'recall'])
  return df

In [12]:
complete_results()

,AUC,F1,recall
bert_textfooler_mcd,0.946972,0.855044,0.822283
bert_textfooler_cov,0.925391,0.803898,0.739674
bert_pwws_mcd,0.943886,0.845557,0.806363
bert_pwws_cov,0.921795,0.793383,0.723533
bert_bae_mcd,0.928870,0.800000,0.733500
bert_bae_cov,0.901882,0.729378,0.631579
bert_tf-adj_mcd,0.923992,0.772636,0.693141
bert_tf-adj_cov,0.878077,0.668122,0.552347
roberta_textfooler_mcd,0.954453,0.882604,0.869154
roberta_textfooler_cov,0.939360,0.856129,0.823962
